In [1]:
# python notebook for Make Your Own Neural Network
# code for a 3-layer neural network, and code for learning the MNIST dataset
# (c) Tariq Rashid, 2016
# license is GPLv2

In [2]:
import numpy
# scipy.special for the sigmoid function expit()
import scipy.special

# helper to load data from PNG image files
import imageio
# glob helps select multiple files using patterns
import glob
# library for plotting arrays
import matplotlib.pyplot
# ensure the plots are inside this notebook, not an external window
%matplotlib inline

from datetime import datetime
startTime = datetime.now()

In [3]:
# neural network class definition
class neuralNetwork:
    
    
    # initialise the neural network
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        # link weight matrices, wih and who
        # weights inside the arrays are w_i_j, where link is from node i to node j in the next layer
        # w11 w21
        # w12 w22 etc 
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

        # learning rate
        self.lr = learningrate
        
        # activation function is the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        pass

    
    # train the neural network
    def train(self, inputs_list, targets_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # hidden layer error is the output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # update the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        # update the weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass

    
    def query(self, inputs_list):
        # convert inputs list to 2d array
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs


In [4]:
input_nodes = 40000
hidden_nodes = 100
output_nodes = 1

learning_rate = 0.1

n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

In [5]:
our_own_dataset = []
    
text = ''
for item in range(3):
    print(item)
    if item == 0:
        text = '????'
    # elif item == 1:
    #     text = '??'
    # elif item == 2:
    #     text = '???'
    # elif item == 3:
    #     text = '????'
    else:
        continue
    for image_file_name in glob.glob('MyTriangle/'+text+'.png'):
        label = 0
        img_array = imageio.imread(image_file_name, as_gray=True)
        img_data  = 255.0 - img_array.reshape(10000)
        record = numpy.append(label,img_data)
        our_own_dataset.append(record)
        pass
print(len(our_own_dataset))

0
1
2
20000


In [6]:
epochs = 1

for e in range(epochs):
    for all_values in our_own_dataset:
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        targets = numpy.zeros(output_nodes) + 0.01
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        pass
    pass

In [7]:
our_own_test = []
i = 0
for item in range(3):
    print(item)
    if item == 0:
        text = '?'
    elif item == 1:
        text = '??'
    # elif item == 2:
    #     text = '?????'
    else:
        continue
    for image_file_name in glob.glob('MyTriangle/'+text+'.png'):
        i+=1
        label = 0
        img_array = imageio.imread(image_file_name, as_gray=True)
        img_data  = 255.0 - img_array.reshape(10000)
        img_data = (img_data / 255.0 * 0.99) + 0.01
        record = numpy.append(label,img_data)
        if i % 220 == 0:
            our_own_test.append(record)
        pass
    for image_file_name in glob.glob('MyTriangle/'+text+'.png'):
        i+=1
        label = 0
        img_array = imageio.imread(image_file_name, as_gray=True)
        img_data  = 255.0 - img_array.reshape(10000)
        img_data = (img_data / 255.0 * 0.99) + 0.01
        record = numpy.append(label,img_data)
        if i % 220 == 0:
            our_own_test.append(record)
        pass

0
1
2


In [8]:
scorecard = []
for all_values in our_own_test:
    correct_label = int(all_values[0])
    inputs = all_values[1:]
    outputs = n.query(inputs)
    label = numpy.argmax(outputs)
    print(outputs, correct_label, label)
    if (label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)
        pass
    
    pass

[[0.99000012]] 1 0
[[0.99000191]] 1 0
[[0.98999972]] 1 0
[[0.9900019]] 1 0
[[0.99000297]] 1 0
[[0.9900019]] 1 0
[[0.98999914]] 1 0
[[0.98999681]] 1 0
[[0.98999764]] 1 0
[[0.98999943]] 1 0
[[0.98999872]] 1 0
[[0.99000021]] 1 0
[[0.99000273]] 1 0
[[0.98999981]] 1 0
[[0.98999896]] 1 0
[[0.98999796]] 1 0
[[0.99000165]] 1 0
[[0.99000198]] 1 0
[[0.98999991]] 1 0
[[0.9900026]] 1 0
[[0.9899986]] 1 0
[[0.99000304]] 1 0
[[0.99000012]] 1 0
[[0.98999646]] 1 0
[[0.98999944]] 1 0
[[0.98999708]] 1 0
[[0.99000051]] 1 0
[[0.989999]] 1 0
[[0.98999581]] 1 0
[[0.9900026]] 1 0
[[0.99000348]] 1 0
[[0.99000161]] 1 0
[[0.98999467]] 1 0
[[0.98999975]] 1 0
[[0.99000193]] 1 0
[[0.99000098]] 1 0
[[0.98999913]] 1 0
[[0.9899961]] 1 0
[[0.99000089]] 1 0
[[0.98999983]] 1 0
[[0.99000162]] 1 0
[[0.99000252]] 1 0
[[0.98999981]] 1 0
[[0.98999804]] 1 0
[[0.98999949]] 1 0


In [9]:
# calculate the performance score, the fraction of correct answers
scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)
diff_time = datetime.now() - startTime
print("time = ", diff_time)



performance =  0.0
time =  0:01:37.565946
